<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Autoencoders. In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime locally, on AWS SageMaker, on Colab or on a comparable environment. If something is running longer, double check your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a LSTM classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for object detection
* <a href="#p3">Part 3</a>: Describe a use case for an autoencoder
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - LSTMSs

Use a LSTM to fit a multi-class classification model on Reuters news articles to distinguish topics of articles. The data is already encoded properly for use in a LSTM model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well as the LSTM code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [106]:
# Imports
from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.layers import LSTM

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [107]:
#Starter code
from tensorflow.keras.datasets import reuters

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [108]:
#Starter code
word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")
print ('Length of text: {} characters'.format(len(word_index)))

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.
Length of text: 30979 characters


In [110]:
# Encode Data as Chars
text = " ".join(word_index)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [112]:
len(chars)

39

In [114]:
# Create the sequence data
max_features = len(word_index.values()) + 1
step = 5
batch_size = 32
maxlen = 80

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))


sequences:  50323


In [116]:
sequences[0]

[14,
 16,
 23,
 2,
 17,
 3,
 12,
 24,
 4,
 17,
 16,
 0,
 10,
 18,
 29,
 29,
 12,
 17,
 24,
 6,
 6,
 16,
 29,
 17,
 25,
 12,
 13,
 10,
 11,
 13,
 10,
 17,
 2,
 6,
 4,
 12,
 2,
 11,
 1,
 0,
 16,
 17,
 29,
 12,
 9,
 11,
 6,
 13,
 17,
 4,
 25,
 12,
 13,
 9,
 25,
 12,
 23,
 18,
 34,
 11,
 17,
 34,
 0,
 3,
 18,
 13,
 16,
 11,
 13,
 10,
 17,
 25,
 0,
 34,
 14,
 12,
 13,
 13,
 17,
 35]

In [117]:
# Create x & y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [118]:
# Checking shape of x
x.shape

(50323, 80, 39)

In [120]:
# Checking shape of y
y.shape

(50323, 39)

In [127]:
# Create model with summary:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_26 (LSTM)               (None, 128)               86016     
_________________________________________________________________
dense_22 (Dense)             (None, 39)                5031      
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


In [123]:
# Sample function that we used in lecture:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [125]:
# Function to generate text after each epoch
def on_epoch_end(epoch, _):
    '''
    This function generates text after each epoch
    '''
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [128]:
# fit the model

model.fit(x, y,
          batch_size=256, # Default is 32, crank up the speed with 256
          epochs=5,
          callbacks=[print_callback])

Train on 50323 samples
Epoch 1/5
50176/50323 [============================>.] - ETA: 0s - loss: 3.0462 - accuracy: 0.1334
----- Generating text after Epoch: 0
----- Generating with seed: "6864 kay amselco corportation area's loopholes ulcerants popularity unreasonable"
6864 kay amselco corportation area's loopholes ulcerants popularity unreasonablecipapl4siaew  t srsmactaaae y alos sinbenknrs 6dca oit fr osc1 vmtiaism t vne ebc essesno ptepoekgi he eswlaoyss tec oa m6elomenabasiemncnen myon bwwg aabemaahesq1gdvetetbvioreeewirgbtifeanni mnsiioas s' cbggtfaimlgrpanstesarattes s'recmgaogssdwmgocn'ngt' amavk0adkt nahkcacmaiti vaniagtfstartemsnufmnoecbre mm nanuoeiauhillayane 'sunbpaoonsuh  u opoopeny ioeniytt pa ethein es mrdrmsianotresor hviun
50323/50323 [==============================] - 132s 3ms/sample - loss: 3.0460 - accuracy: 0.1335
Epoch 2/5
50176/50323 [============================>.] - ETA: 0s - loss: 2.7752 - accuracy: 0.2032
----- Generating text after Epoch: 1
----- Generating

## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*  The pad_sequences is used to make sure that all the sequences in the list have the same length.  If you don't use it you won't have the same size arrays and it won't work so pad_sequences adds zeros



## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*
An lstm is a rnn with a memory compenent to it and the advantage is that lstms are able to maintain and "remember" information.  A lstm model can put more weight on the short term memory while retaining info from the long term.


## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*
1) A good use case is when you're working with stock prices because They are really good at processing series data.
2) lstms are good at predicting text as in the case above with reuters.  The reason they are able to do this is because they have the memory component.
3) They are good with weather, again because these models are good at handling time series data.


<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>

In [129]:
#Starter Code
from skimage.io import imread_collection
from skimage.transform import resize #This might be a helpful function for you

images = imread_collection('./frog_images/*.jpg')

In [130]:
#Starter Code
print(type(images))
print(type(images[0]), end="\n\n")

print("Each of the Images is a Different Size")
print(images[0].shape)
print(images[1].shape)

<class 'skimage.io.collection.ImageCollection'>
<class 'imageio.core.util.Array'>

Each of the Images is a Different Size
(2137, 1710, 3)
(3810, 2856, 3)


Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model. Print out the predictions in any way you see fit. 

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - Check for other things such as fish.

# Since the objective here is to print the predictions any way I see fit, I'll simply run the model on a photo that obviously does not have a frog and then run the model on one that obviously does have a frog in it.  See results below!

In [137]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  print(results)
  for entry in results:
    if entry[1] == 'bullfrog':
      return entry[2]
  return 0.0

In [138]:
# This prediction says no on the frog - good because there's no frog here!
img_contains_frog(process_img_path('./frog_images/cristiane-teston-bcnfJvEYm1Y-unsplash.jpg'))

[('n07718747', 'artichoke', 0.30643585), ('n02281787', 'lycaenid', 0.23586062), ('n07730033', 'cardoon', 0.18358354)]


0.0

In [133]:
# This prediction predicts 54.8% chance it's a tree_frog, 30.9% it's a bullfrog, and 14.1% chance it's a tailed_frog
# Good job model!
img_contains_frog(process_img_path('./frog_images/jared-evans-VgRnolD7OIw-unsplash.jpg'))

[('n01644373', 'tree_frog', 0.5484332), ('n01641577', 'bullfrog', 0.3099555), ('n01644900', 'tailed_frog', 0.14135446)]


0.3099555

# Stretch goal to check for a fish:

In [139]:
# Stretch goal: check for fish

def img_contains_fish(img):
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  model = ResNet50(weights='imagenet')
  features = model.predict(x)
  results = decode_predictions(features, top=3)[0]
  print(results)
  for entry in results:
    if entry[1] == 'fish':
      return entry[2]
  return 0.0

In [140]:
# Stretch goal, checking for fish.  There's an 18% chance that this picture (of an actual frog) is
# a hippopotamus!!!!!  Fantastic!  But it's not a fish.  This correctly labels this as a bullfrog with 75% probability
img_contains_fish(process_img_path('./frog_images/matthew-kosloski-sYkr-M78H6w-unsplash.jpg'))

[('n01641577', 'bullfrog', 0.7561675), ('n02398521', 'hippopotamus', 0.18219742), ('n01644900', 'tailed_frog', 0.0299511)]


0.0

<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

One of the coolest use cases for an autoencoder was the example we used in class with wayfare for reverse image search.

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?  Probably my intuition to identify and solve problems.  I do this every day as an entreprenuer and I think that I can bring that ability of mine to the datascience world in a business setting and do fairly well with it.
- What area of Data Science would you most like to learn more about, and why?  The part that I'd like to learn most about is NLP.  I've enjoyed studying NLP probably more than anything because there is an unbelievealbe amount of text data in the world that is very hard to interpret unless you have an efficient way of doing it.  That's where NLP comes in!
- Where do you think Data Science will be in 5 years?  I'm willing to bet that data science will sneak its way into everything.  I know that there are so many usecases for businesses who can afford to pay data scientists to analyze their data but there are a ton of small businesses that don't have that luxury.  My bet is that there will be basic models built for commercial use that the every day user will be able to apply to their situation.  All in all, I think predictive modeling will become very common for a lot of people.
- What are the threats posed by AI to our society?  Honestly, the largest one that I'm concerned about pertains to those in poverty.  I know, from our reading assignments the other day, that poor people tend to have the highest fears related to developing AI.  It's likely that the easiest and lowest paying jobs will be come automated and if that's the case then there wwould be a large % of the population who cannot find jobs.  Time will tell if that's true or not!
- How do you think we can counteract those threats? A few things we can do... First, educate people.  Second, be conscientious of how tech developments will affect those in poverty and not strictly make new AI developments on the basis of monetary value ALONE.  
- Do you think achieving General Artifical Intelligence is ever possible?  I'm not sure to be honest.  I go back in forth.  If it's possible, I believe we are a very long way off.

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [1]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")